<a href="https://colab.research.google.com/github/annikarichardson2024/ChatBot/blob/main/ChatbotEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [26]:
#load data
questions = ['Hello', 'How are you?', 'What is your name?','Goodbye']
answers = ['Hey there!','Very well.', 'My name is bot.','Goodbye!']


In [27]:
# create tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)

In [28]:
# create sequences
sequences_q = tokenizer.texts_to_sequences(questions)
sequences_a = tokenizer.texts_to_sequences(answers)

In [29]:
# Pad the sequences
max_length = max(max(len(seq) for seq in sequences_q), max(len(seq) for seq in sequences_a))
padded_q = tf.keras.preprocessing.sequence.pad_sequences(sequences_q, maxlen=max_length, padding='post')
padded_a = tf.keras.preprocessing.sequence.pad_sequences(sequences_a, maxlen=max_length, padding='post')



In [ ]:
# setparameters
embedding_dim = 2000
units = 1024
vocab_size = 5000


In [31]:
# encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [32]:
# seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile model
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [33]:
decoder_input_data = np.zeros_like(padded_a)
decoder_input_data[:, 0] = 1

# Train  model
num_epochs = 100
model.fit([padded_q, decoder_input_data], np.expand_dims(padded_a, -1), batch_size=2, epochs=num_epochs)


Epoch 1/100
2/2 [==============================] - 3s 68ms/step - loss: 8.5177 - accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 68ms/step - loss: 8.4751 - accuracy: 0.1875
Epoch 3/100
2/2 [==============================] - 0s 16ms/step - loss: 8.4183 - accuracy: 0.1250
Epoch 4/100
2/2 [==============================] - 0s 17ms/step - loss: 8.3070 - accuracy: 0.1250
Epoch 5/100
2/2 [==============================] - 0s 15ms/step - loss: 8.0136 - accuracy: 0.1250
Epoch 6/100
2/2 [==============================] - 0s 15ms/step - loss: 6.7753 - accuracy: 0.1250
Epoch 7/100
2/2 [==============================] - 0s 16ms/step - loss: 4.3452 - accuracy: 0.1250
Epoch 8/100
2/2 [==============================] - 0s 16ms/step - loss: 3.3877 - accuracy: 0.1250
Epoch 9/100
2/2 [==============================] - 0s 14ms/step - loss: 3.0176 - accuracy: 0.1875
Epoch 10/100
2/2 [==============================] - 0s 15ms/step - loss: 2.6084 - accuracy: 0.2500
Epoch 11/100
2/

In [34]:
# function to predict answers
def preprocess_input_text(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate_response(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1  # start token
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:  # end token
            break
    return response_sequence

def sequence_to_text(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

In [35]:
# putting it all together for responses

def chat_with_bot(input_text):
    input_sequence = preprocess_input_text(input_text)
    response_sequence = generate_response(input_sequence)
    response_text = sequence_to_text(response_sequence[0])
    return response_text

In [ ]:
# Interactive chat with the bot
print("Start chatting! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = chat_with_bot(user_input)
    print(f"Bot: {response}")



Start chatting! Type 'quit' to exit.
You: Hello
1/1 [==============================] - 0s 20ms/step
Bot: hey there
You: How are you?
1/1 [==============================] - 0s 20ms/step
Bot: i am doing
